In [64]:
import pandas as pd
from bs4 import BeautifulSoup
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pymongo
from flask import Flask, jsonify
import time

In [30]:
#setup splinter
executable_path= {'executable_path': ChromeDriverManager().install()}
browser= Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/Users/juliankauffmann/.wdm/drivers/chromedriver/mac64/91.0.4472.19/chromedriver] found in cache


In [3]:
#function to have beautifulSoup parse url
def parser (theurl):
    
    #pull up url
    browser.visit(theurl)
    html = browser.html
    thesoup = BeautifulSoup(html, 'html.parser')

    return thesoup

In [4]:
#1)Scrape the Mars News Site and collect the latest News Title and Paragraph Text
url='https://redplanetscience.com/'
soup=parser(url)

#declare variables
mars_titles=[]
mars_paragraphs=[]

#scrape titles
results=soup.find_all('div', class_='content_title')

#append titles to list
for result in results:
    mars_titles.append(result.text.strip())

#print results to verify
mars_titles

["NASA's New Mars Rover Is Ready for Space Lasers",
 'NASA Establishes Board to Initially Review Mars Sample Return Plans',
 'Two Rovers to Roll on Mars Again: Curiosity and Mars 2020',
 'Naming a NASA Mars Rover Can Change Your Life',
 "NASA's Mars Perseverance Rover Gets Its Sample Handling System",
 "NASA's Mars Helicopter Attached to Mars 2020 Rover",
 "NASA's Perseverance Rover Will Carry First Spacesuit Materials to Mars",
 "Curiosity Mars Rover's Summer Road Trip Has Begun",
 'NASA to Hold Mars 2020 Perseverance Rover Launch Briefing',
 "NASA's Mars 2020 Rover Completes Its First Drive",
 "How NASA's Perseverance Mars Team Adjusted to Work in the Time of Coronavirus",
 "NASA's Mars Perseverance Rover Passes Flight Readiness Review",
 "Meet the People Behind NASA's Perseverance Rover",
 "NASA's Perseverance Mars Rover Gets Its Wheels and Air Brakes",
 "Media Get a Close-Up of NASA's Mars 2020 Rover"]

In [5]:
#scrape paragraph
results=soup.find_all('div', 'article_teaser_body')

#append paragraphs to list
for result in results:
    mars_paragraphs.append(result.text.strip())
    
#print 1st paragraph to verify it worked
print(mars_paragraphs[0])

In [9]:
#2)Visit the url for the Featured Space Image site here.

#parse feature image site
url='https://spaceimages-mars.com/'
soup=parser(url)

In [10]:
#Use splinter to navigate the site and find the image url for the current Featured Mars Image
browser.links.find_by_partial_text('FULL IMAGE').click()

#store new html
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

#save feature image source
result=soup.find('img', class_='fancybox-image')
featured_image_url=url+result['src']

print(featured_image_url)

In [20]:
#Visit the Mars Facts webpage here and use Pandas to scrape the table

url='https://galaxyfacts-mars.com/'
tables=pd.read_html(url)

In [27]:
#Use Pandas to convert the data to a HTML table string.
the_table=tables[1].set_index(0)
the_table_html=the_table.to_html()


In [81]:
#function to pull the image url
def imagePull(hemisphere):
    
    #click on link for hemisphere
    browser.links.find_by_partial_text(hemisphere).click()
    
    #parse the new browser
    html = browser.html
    temp = BeautifulSoup(html, 'html.parser')
    
    #find the large image file
    the_image=temp.find('a',text='Sample')
    
    #move back to original page
    browser.links.find_by_partial_text('Back').click()

    return the_image['href']

#----------------------------------------------------
#list to store dictionary of image url and title
hemisphere_image_urls=[]

#open the page
url='https://marshemispheres.com/'
soup=parser(url)

#finds link names
links=soup.find_all('h3')

#loops through the first four links
for link in links:
    if link.text !='Back':
        the_image_url=f'{url}{imagePull(link.text)}'
        
        #append the list with a dictionary
        hemisphere_image_urls.append({"title": link.text, "img_url":the_image_url})

hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg'}]

In [ ]:
browser.quit()